In [33]:
import numpy as np  
import pandas as pd
import random  
import string
import bs4 as bs  
import urllib.request  
import re
from collections import Counter

import tweepy
import TwitterCredentials as keys

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vpathalam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [26]:
auth = tweepy.OAuthHandler(keys.consumer_API_key, keys.consumer_API_secret_key)
auth.set_access_token(keys.access_token, keys.access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

screen_name = "wsj"
alltweets = []  
new_tweets = api.user_timeline(screen_name = screen_name,count=200)
alltweets.extend(new_tweets)
oldest = alltweets[-1].id - 1

while len(new_tweets) > 0:
    print(f"getting tweets before {oldest}")        
    new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)  
    alltweets.extend(new_tweets)
    oldest = alltweets[-1].id - 1
    print(f"...{len(alltweets)} tweets downloaded so far")

getting tweets before 1334769458490077183
...400 tweets downloaded so far
getting tweets before 1333962750381486079
...600 tweets downloaded so far
getting tweets before 1333150005595680767
...800 tweets downloaded so far
getting tweets before 1332210074073378816
...1000 tweets downloaded so far
getting tweets before 1331394712046014464
...1200 tweets downloaded so far
getting tweets before 1330617134960009218
...1400 tweets downloaded so far
getting tweets before 1329760170931531778
...1600 tweets downloaded so far
getting tweets before 1328948577993957376
...1800 tweets downloaded so far
getting tweets before 1328189842761670655
...2000 tweets downloaded so far
getting tweets before 1327344267187150855
...2200 tweets downloaded so far
getting tweets before 1326570387535245311
...2400 tweets downloaded so far
getting tweets before 1325785215093018623
...2600 tweets downloaded so far
getting tweets before 1324883047116894207
...2800 tweets downloaded so far
getting tweets before 132418

In [27]:
df = pd.DataFrame({"realDonaldTrump": alltweets})
df

,realDonaldTrump
0,Status(_api=<tweepy.api.API object at 0x7fac9e...
1,Status(_api=<tweepy.api.API object at 0x7fac9e...
2,Status(_api=<tweepy.api.API object at 0x7fac9e...
3,Status(_api=<tweepy.api.API object at 0x7fac9e...
4,Status(_api=<tweepy.api.API object at 0x7fac9e...
...,...
3233,Status(_api=<tweepy.api.API object at 0x7fac9e...
3234,Status(_api=<tweepy.api.API object at 0x7fac9e...
3235,Status(_api=<tweepy.api.API object at 0x7fac9e...
3236,Status(_api=<tweepy.api.API object at 0x7fac9e...


In [28]:
#function to convert the tweepy object to text
def totext(tweet):
    return tweet.text
#define new data frame to hold texts of tweets
dfnew = pd.DataFrame({"realDonaldTrump": []})
#map above function to each column in df
dfnew["realDonaldTrump"] = df["realDonaldTrump"].map(totext)
dfnew

,realDonaldTrump
0,A government watchdog agency found no wrongdoi...
1,“We have no conversation with other residents....
2,Google is deciding whether to impose severe pe...
3,Trump’s musings about running again in 2024 ar...
4,From @WSJopinion: Continued gridlock would be ...
...,...
3233,It feels harder than ever to interact with peo...
3234,"Would you buy a $2,000 dress on Amazon? That’s..."
3235,"New jobless claims fell to a seven-month low, ..."
3236,"For major sports leagues, the weirdest part of..."


In [29]:
tot = set()
all_topics = ["job", "economy", "trade", "market", "stock", "DJIA", "NASDAQ", "price", "jobs", "employed", "unemployed", "employment", "unemployment", "stock exchange", "bull", "bear", "market", "new york stock exchange", "shares", "financials", "taxes", "companies", "tax", "economic"]
for idx, row in dfnew.iterrows():
    corpus = nltk.sent_tokenize(row.values[0])
    for i in range(len(corpus)):
        corpus[i] = corpus[i].lower()
        corpus[i] = re.sub(r'\W',' ',corpus[i])
        corpus[i] = re.sub(r'\s+',' ',corpus[i])
        corpus[i] = re.sub(r"http\S+", "", corpus[i])
        corpus[i] = re.sub(r'\w*\d\w*', '', corpus[i]).strip()
        for topic in all_topics:
            if topic in corpus[i]:
                tot.add(corpus[i])
print(len(tot))

371


In [30]:
wordfreq = {}
for sentence in tot:
    tokens = nltk.word_tokenize(sentence)
    for token in tokens:
        if token not in wordfreq.keys():
            wordfreq[token] = 1
        else:
            wordfreq[token] += 1

In [31]:
print(stopwords.words('english'))

for w in set(stopwords.words('english'))  :
    if str(w) in wordfreq:
        del wordfreq[str(w)]
del wordfreq['co']
del wordfreq['u']
#del wordfreq['trump']
#del wordfreq['wsj']
#del wordfreq['wsjwhatsnow']
len(wordfreq)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

1889

In [34]:
d = Counter(wordfreq)
newdict = {}
for k, v in d.most_common(20):
    print('%s: %i' % (k, v))
    newdict[k] = v

companies: 73
market: 46
stocks: 36
economy: 32
economic: 31
pandemic: 31
stock: 29
new: 27
covid: 24
president: 23
biden: 23
trade: 23
markets: 20
china: 19
joe: 19
jobs: 19
year: 18
week: 17
shares: 17
wsjwhatsnow: 16


In [ ]:
plt.bar(range(len(newdict)), list(newdict.values()), align='center')
plt.xticks(range(len(newdict)), list(newdict.keys()))
plt.xticks(rotation=45)
plt.show()
#plt.savefig('commonwords2.png')

In [ ]:
wc = WordCloud(background_color="white",width=1000,height=1000, max_words=20,relative_scaling=0.5,normalize_plurals=False).generate_from_frequencies(wordfreq)
plt.imshow(wc)
#plt.savefig('wordcloud.png')

In [44]:
import csv
with open("./kaggle_dataset.csv", "r", encoding = "ISO-8859-1") as f:
    reader = csv.reader(f)
    i = next(reader)
    rest = list(reader)
    
i

['target', 'ids', 'date', 'flag', 'user', 'text']

In [45]:
data = pd.read_csv("kaggle_dataset.csv", encoding = "ISO-8859-1", engine='python')
target = data["target"].astype(float)
text = data["text"].values
tfv=TfidfVectorizer(min_df=0, max_features=3000, strip_accents='unicode',lowercase =True,
                            analyzer='word', token_pattern=r'\w{3,}', ngram_range=(1,1),
                            use_idf=True,smooth_idf=True, sublinear_tf=True, stop_words = "english")   
data=tfv.fit_transform(text)
print(data.shape)

(1600000, 3000)
